# Agentic Search tool

In [1]:
# old
import os
TAVILY_API_KEY = "Your API KEY"
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
from tavily import TavilyClient

# Step 1. Instantiating your TavilyClient
tavily = TavilyClient(api_key=TAVILY_API_KEY)


# Lets try to create a POST writer Agent

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
# from langchain_ollama import ChatOllama
import json
memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
model = OllamaFunctions(model="llama3.1", format="json")

In [4]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [5]:
PLAN_PROMPT = """
Act as an expert Instagram news post planner. 
Create a comprehensive outline for an Instagram post based on the given topic. 
Your outline should include: 
* Core message and key points 
* Engaging headline and captivating introduction
* Strong conclusion that leaves a lasting impression 
* Visual suggestions (images, videos, graphics) 
* Relevant hashtags and a compelling caption
* Potential call-to-actions to drive engagement
* Recommended tone and style for the post 

Provide clear and concise explanations for each section of the outline. 
"""

WRITER_PROMPT = """
Assume the role of a skilled Instagram news post writer. 
Craft a captivating and informative Instagram post based on the provided topic and outline. 
Ensure the content aligns with the target audience and platform's style guidelines.

Utilize the provided information (if any) to enrich your post. 
Be prepared to revise the post based on user feedback. 

Focus on creating engaging content that resonates with the Instagram community. 
------

{content}
"""

REFLECTION_PROMPT = """
Assume the role of an expert Instagram content evaluator.
Provide constructive feedback on the provided Instagram post, focusing on:
* Engagement potential: How well does the post capture attention and interest?
* Clarity and conciseness: Is the message clear and easy to understand?
* Visual appeal: How effective are the images or videos?
* Tone and style: Does the post align with the target audience and platform?
* Overall impact: How well does the post achieve its intended goals? 

Offer specific suggestions for improvement, including recommendations for length, depth, style, and visual elements. 
"""

RESEARCH_PLAN_PROMPT = """
Act as a dedicated research analyst. 
Develop three concise search queries to gather essential information for an Instagram news post on the given topic. 
Prioritize queries that yield relevant and up-to-date data. 
Consider using specific keywords and phrases for accurate results. 
"""

RESEARCH_CRITIQUE_PROMPT = """
Act as a dedicated research analyst. 
Develop three focused search queries to gather data for revising an Instagram post based on the given feedback. 
Prioritize queries that address the specific areas needing improvement. 
Ensure the search terms align with the post's original topic and the provided critiques. 
"""

In [6]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [7]:
def plan_node(state: AgentState):
    print("research_plan_node-state_task", state['task'])
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    print("Resrch Plan Node ",queries)
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [8]:
builder = StateGraph(AgentState)

builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("planner")

builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [9]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [10]:
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(checkpointer=checkpointer)


In [17]:
thread = {"configurable": {"thread_id": "1"}}
# builder.graph.get_state(thread)
for s in graph.stream({
    'task': "all updates on olympics 2024",
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)


research_plan_node-state_task all updates on olympics 2024
{'planner': {'plan': "Here's a comprehensive outline for an Instagram post based on Olympics 2024:\n\n**Core Message and Key Points:**\n- Update on upcoming Olympic Games in Paris, France (2024)\n- Schedule of key events and competitions\n- Preview of participating countries and athletes\n\n**Engaging Headline and Captivating Introduction:***'Get Ready for the Ultimate Competition!'*\n*Introduction:* 'The Olympic spirit is back! With just a few months to go, we're counting down the days until the 2024 Olympics in Paris. From record-breaking feats to unforgettable moments of triumph, this year's Games promise to be an unmissable experience.\n\n**Strong Conclusion that Leaves a Lasting Impression:**\n*Conclusion:* 'Don't miss out on the magic! Mark your calendars for July 26th and get ready to cheer on your favorite athletes. Stay tuned for more updates, and let us know who you're rooting for in the comments below!\n\n**Visual Su

In [ ]:
states = graph.get_state(config=thread)

In [31]:
states.values

{'task': 'all updates on olympics 2024',
 'plan': "Here's a comprehensive outline for an Instagram post based on Olympics 2024:\n\n**Core Message and Key Points:**\n- Update on upcoming Olympic Games in Paris, France (2024)\n- Schedule of key events and competitions\n- Preview of participating countries and athletes\n\n**Engaging Headline and Captivating Introduction:***'Get Ready for the Ultimate Competition!'*\n*Introduction:* 'The Olympic spirit is back! With just a few months to go, we're counting down the days until the 2024 Olympics in Paris. From record-breaking feats to unforgettable moments of triumph, this year's Games promise to be an unmissable experience.\n\n**Strong Conclusion that Leaves a Lasting Impression:**\n*Conclusion:* 'Don't miss out on the magic! Mark your calendars for July 26th and get ready to cheer on your favorite athletes. Stay tuned for more updates, and let us know who you're rooting for in the comments below!\n\n**Visual Suggestions (Images, Videos, Gra

In [29]:
print(states.values.get("plan"))

Here's a comprehensive outline for an Instagram post based on Olympics 2024:

**Core Message and Key Points:**
- Update on upcoming Olympic Games in Paris, France (2024)
- Schedule of key events and competitions
- Preview of participating countries and athletes

**Engaging Headline and Captivating Introduction:***'Get Ready for the Ultimate Competition!'*
*Introduction:* 'The Olympic spirit is back! With just a few months to go, we're counting down the days until the 2024 Olympics in Paris. From record-breaking feats to unforgettable moments of triumph, this year's Games promise to be an unmissable experience.

**Strong Conclusion that Leaves a Lasting Impression:**
*Conclusion:* 'Don't miss out on the magic! Mark your calendars for July 26th and get ready to cheer on your favorite athletes. Stay tuned for more updates, and let us know who you're rooting for in the comments below!

**Visual Suggestions (Images, Videos, Graphics):***
- A dramatic image of the Eiffel Tower with a hint of

In [19]:
print(s.get('plan'))

None


In [1]:
from graphviz import Digraph

# Create a new directed graph
dot = Digraph()

# Add nodes (these are the states)
dot.node("planner", "Planner")
dot.node("research_plan", "Research Plan")
dot.node("generate", "Generate")
dot.node("reflect", "Reflect")
dot.node("research_critique", "Research Critique")
dot.node("image_generate", "Image Generate")
dot.node("end", "END")

# Add edges with labels to represent the flow
dot.edge("planner", "research_plan")
dot.edge("research_plan", "generate")
dot.edge("generate", "reflect", label="if revisions < max")
dot.edge("generate", "image_generate", label="if revisions == max and images exist")
dot.edge("generate", "end", label="if revisions == max and no images")
dot.edge("reflect", "research_critique")
dot.edge("research_critique", "generate")

# Render and visualize the graph
dot.render("state_graph", format="png", cleanup=True)


'state_graph.png'

# Images

In [15]:
IMAGE_PROMPT_GENERATOR ="""Act as a Prompt Engineer. Analyze the user's input text, identify the context, key elements, and desired visuals. 
 Develop three concise prompts for AI text to image model . Ensure the prompts are specific, clear, and comprehensive 
 for accurate image generation.
 """

import os
import requests
import json, io
from PIL import Image
import base64

def generate_image(prompt, api_key="YOUR API KEY", model_name="diffusion1XL", image_height=1024, image_width=1024,
                   negative_prompt=None, num_output_images=1, guidance_scale=7.5, num_inference_steps=10, seed=None, output_img_type="pil"):
    payload = {
        "modelName": model_name,  # DO NOT CHANGE THIS VALUE
        "prompt": prompt,
        "prompt2": "",  # Optional - Specify any additional prompt details if required
        "imageHeight": image_height,  # 1024 gives the best output. Modify this based on your requirement.
        "imageWidth": image_width,  # 1024 gives the best output. Modify this based on your requirement.
        "negativePrompt": negative_prompt,  # Optional, mention aspects to be ignored in the output image
        "negativePrompt2": None,  # Optional, mention aspects to be ignored in the output image
        "numOutputImages": num_output_images,  # Optional, Defaults to 1. Max value is 5
        "guidanceScale": guidance_scale,  # Optional, Defaults to 5. Optimal values are < 15.
        "numInferenceSteps": num_inference_steps,  # Optional, Defaults to 50. Max value is 100.
        "seed": seed,  # Optional - Leave it empty to generate a random value.
        "outputImgType": output_img_type  # Optional, Defaults to pil. Supported: pil, latent.
    }

    headers = {"content-type": "application/json", "Authorization": f"Bearer {api_key}"}  # API Key

    url = "https://cloud.olakrutrim.com/v1/images/generations/diffusion"
    resp = requests.post(url, headers=headers, json=payload)

    # Check if the response is successful
    if resp.status_code == 200:
        output = resp.json()
        img_datas = output["data"]
        for index, img_data in enumerate(img_datas):
            # Decode the base64 string to bytes
            img_bytes = base64.b64decode(img_data["b64_json"])

            # Convert bytes to a PIL image
            img = Image.open(io.BytesIO(img_bytes))

            # Save the image locally (optional)
            img.save(f"generated_image_{index}.png")

            # Display the image
            img.show()

    else:
        print("Failed to get a response from the server. Status code:", resp.status_code)
        print("Response content:", resp.text)



def prompt_plan_node(states):
    
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=IMAGE_PROMPT_GENERATOR),
        HumanMessage(content=states.values.get("task"))
    ])
    print("Prompts Generated: ", queries.queries)
    
    image_paths = []
    
    for q in queries.queries:
       
        # Generate image using the prompt
        image_path = generate_image(
            prompt=q,
            image_height=1024,  # Adjust these parameters as needed
            image_width=1024
        )
        image_paths.append(image_path)
    
    
    return {"image_paths": image_paths}





In [ ]:
# result = prompt_plan_node(states)

# Gradio

In [11]:
import gradio as gr

In [13]:
# Gradio Interface
def run_workflow(task, max_revisions):
    state = {
        'task': task,
        "max_revisions": max_revisions,
        "revision_number": 1,
    }
    thread = {"configurable": {"thread_id": "1"}}
    outputs = []
    with SqliteSaver.from_conn_string(":memory:") as checkpointer:
        graph = builder.compile(checkpointer=checkpointer)
        for s in graph.stream(state, thread):
            outputs.append(s)
    states = graph.get_state(config=thread)
    return states

def display_results(states_):
    # states = graph.get_state(config=thread)
    plan = states_.values.get("plan")
    draft = states_.values.get("draft")
    critique = states_.values.get("critique")
    return plan, draft, critique

# Set up Gradio inputs and outputs
task_input = gr.Textbox(label="Instagram Post Topic")
max_revisions_input = gr.Slider(1, 5, step=1, label="Max Revisions", value=2)
output_plan = gr.Textbox(label="Plan")
output_draft = gr.Textbox(label="Draft")
output_critique = gr.Textbox(label="Critique")

# Create Gradio interface
iface = gr.Interface(
    fn=lambda task, max_revisions: display_results(run_workflow(task, max_revisions)),
    inputs=[task_input, max_revisions_input],
    outputs=[output_plan, output_draft, output_critique],
    title="Instagram Post Generator",
    description="Generate and refine Instagram posts using AI."
)

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


research_plan_node-state_task How to Grow Your Personal Brand on LinkedIn
Resrch Plan Node  queries=['Personal branding strategies for LinkedIn professionals', 'Effective ways to increase online presence on LinkedIn', 'Tips for creating engaging content and profiles on LinkedIn']
